# Tensorflow를 이용한 MNIST 실습 

## 핵심은 정확도 비교해보기!
#### 1) non-linearity 바꿔보기 : sigmoid -> relu
#### 2) weight 초기화 바꿔보기: random -> xavier, he

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
tf.reset_default_graph()

In [ ]:
mnist = input_data.read_data_sets('data/mnist/', one_hot=True)

In [3]:
mnist.train, mnist.test

NameError: name 'mnist' is not defined

ERROR! Session/line number was not unique in database. History logging moved to new session 171


In [ ]:
X = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

neural networks 구축  
1) weight 초기화 바꿔보기  
2) non-linearity 바꿔보기

In [ ]:
W_1 = tf.get_variable("w1", shape=[784, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b_1 = tf.get_variable("b1", shape=[128], 
                     initializer=tf.contrib.layers.xavier_initializer())
hidden_1  = tf.nn.relu(tf.matmul(X, W_1) + b_1)

W_2 = tf.get_variable("w3", shape=[128, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b_2 =  tf.get_variable("b3", shape=[10], 
                     initializer=tf.contrib.layers.xavier_initializer())
hypo = tf.matmul(hidden_1 , W_2) + b_2

#### 초기화와 non-linearity의 상관관계
1) sigmoid = xavier  
2) relu = he : tf.contrib.layers.variance_scaling_initializer()

In [63]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        logits=hypo, labels=y))
learning_rate = 0.1
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [64]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        logits=hypo, labels=tf.stop_gradient(y)))

In [65]:
learning_rate = 1
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [66]:
predict = tf.cast(hypo > 0.5, tf.float32) # 1 or 0
acc = tf.reduce_mean(tf.cast(tf.equal(predict, y), tf.float32))

In [67]:
batch_size = 100
epoches = 10

In [68]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # total_batch 수(업데이트 수) = 전체 데이터 개수 // batch크기
    total_batch = mnist.train.num_examples//batch_size
    print('total_batch : ', total_batch)

    for epoch in range(epoches):
        avg_cost = 0

        for i in range(total_batch):
            #training dataset
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)


            sess.run(optimizer, 
                     feed_dict={X: batch_xs, y: batch_ys})
            c = sess.run(cost
                         , feed_dict={X: batch_xs, y: batch_ys})

            avg_cost += c / total_batch

        print('{:2} : {:}'.format(epoch+1, avg_cost))
    print('Learning finished!')
    
    pred = tf.equal(tf.argmax(hypo, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(pred, tf.float32))

    print('accuracy :', sess.run(accuracy, feed_dict={X: mnist.test.images, 
                                                  y: mnist.test.labels}))


total_batch :  550
 1 : 0.24340792115100385
 2 : 0.052112699679353024
 3 : 0.032282024928453286
 4 : 0.02396197996271604
 5 : 0.01868721660268915
 6 : 0.014412688048217768
 7 : 0.011446066415149036
 8 : 0.010679033734539357
 9 : 0.00826593230160969
10 : 0.007479154927952384
Learning finished!
accuracy : 0.971
